In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rand
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
data = pd.read_csv("IRIS.csv")
data.head(6)
data.isnull().sum()

import pandas_profiling as pp
#report = pp.ProfileReport(data)
#report.to_file("report.html")
#report

# Without combining any classes

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ShuffleSplit


### Using the bagging classifier
From Kaggle kernel
1. File->Add/Upload Data
2. Select the Utility Scripts dataset (Your Datasets->Utility Scripts)
(Make sure to restart kernel whenever changes have been made to the scripts)

In [4]:
import sys
sys.path.append('/kaggle/input/utility-scripts/')
from evaluate import *
from util import *

## Single Classifiers

In [5]:
feats = [x for x in data.columns if x != "species"]
sss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
X = data[feats]
y = data["species"]

In [6]:
acc_score = dict()
models = {'Logistic Regression': LogisticRegression(), 'LDA': LinearDiscriminantAnalysis(),
         'kNN':KNeighborsClassifier(n_neighbors=1), 'DT':DecisionTreeClassifier(), 'SVM':SVC()}
model_params = {'Logistic Regression': {'solver': 'lbfgs', 'multi_class':'auto', 'max_iter': 200}, 
                'SVM':{'gamma':'auto'}, 'LDA': {}, 'kNN':{}, 'DT':{}}

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    for model_name, model_fn in models.items():
        if model_name not in acc_score.keys():
            acc_score[model_name] = []
        curr_model = model_fn
        curr_model.set_params(**model_params[model_name])
        curr_model.fit(X_train, y_train)
        acc_score[model_name].append(metrics.accuracy_score(y_test, curr_model.predict(X_test)))

acc_score
for key, val in acc_score.items():
    print(key +": " + str(np.mean(val)))

Logistic Regression: 0.9666666666666668
LDA: 1.0
kNN: 0.9733333333333334
DT: 0.9466666666666667
SVM: 0.9733333333333334


## Bagged Classifier

In [7]:
oob_bagged = dict()
for i in range(1,6):
    for model_name, model_fn in models.items():
        if(model_name not in oob_bagged.keys()):
            oob_bagged[model_name] = []
        errors = calculate_oob(data[feats], data["species"], model_fn, model_params[model_name], seed=i)
        oob_bagged[model_name].append(errors)
        print(model_name + " [train oob: " + str(errors['train oob']) + ", test oob: " + str(errors['test oob']) + "]")



Logistic Regression [train oob: 0.0, test oob: 0.0]
LDA [train oob: 0.0, test oob: 0.0]


KeyboardInterrupt: 

In [8]:

test_bagged = dict()
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    for model_name, model_fn in models.items():
        if model_name not in test_bagged.keys():
            test_bagged[model_name] = []
        predictor_fn, Models, not_seen, seen_data = create_bagged_predictor(X_train, y_train, model_fn, model_params[model_name], X_train.shape[1])
        print(X_test)
        pred, freq = predictor_fn(X_test)
        test_bagged[model_name].append(metrics.accuracy_score(y_test, pred))

print(test_bagged)

     sepal_length  sepal_width  petal_length  petal_width
114           5.8          2.8           5.1          2.4
62            6.0          2.2           4.0          1.0
33            5.5          4.2           1.4          0.2
107           7.3          2.9           6.3          1.8
7             5.0          3.4           1.5          0.2
100           6.3          3.3           6.0          2.5
40            5.0          3.5           1.3          0.3
86            6.7          3.1           4.7          1.5
76            6.8          2.8           4.8          1.4
71            6.1          2.8           4.0          1.3
134           6.1          2.6           5.6          1.4
51            6.4          3.2           4.5          1.5
73            6.1          2.8           4.7          1.2
54            6.5          2.8           4.6          1.5
63            6.1          2.9           4.7          1.4
37            4.9          3.1           1.5          0.1
78            

ValueError: too many values to unpack (expected 2)

In [9]:
predictor_fn(X_test)

[ModeResult(mode=array([['Iris-virginica']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-versicolor']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-setosa']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-virginica']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-setosa']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-virginica']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-setosa']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-versicolor']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-versicolor']], dtype=object), count=array([[49]])),
 ModeResult(mode=array([['Iris-versicolor']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-virginica']], dtype=object), count=array([[50]])),
 ModeResult(mode=array([['Iris-versicolor']], dtype=object), count=array([[50]])),
 ModeResult(mode=arr